![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
LangChain 101
</b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()
# Bei Bedarf: Trennen zwischen Installationsname () und Importname (für Python) beide Angaben in Klammern
# install_packages([('markitdown[all]', 'markitdown'), 'langchain_chroma', ]

# 1 | Was ist LangChain?
---

LangChain ist ein Framework zur **Entwicklung von Anwendungen mit großen Sprachmodellen** (LLMs). Es vereinfacht die Integration von LLMs in eigene Anwendungen durch:

- Modulare Komponenten für verschiedene Aufgaben
- Vorgefertigte Ketten (Chains) für komplexe Workflows
- Einfache Integration externer Datenquellen
- Werkzeuge für das Speichern von Konversationskontexten



<img src="https://python.langchain.com/img/brand/wordmark.png" class="logo" width="360"/>



[Einführung](https://python.langchain.com/docs/introduction/)   
[Konzepte](https://python.langchain.com/docs/concepts/)   
[API-References](https://python.langchain.com/api_reference/index.html)   
[Integrations](https://python.langchain.com/docs/integrations/providers/)

---

[Tutorials](https://python.langchain.com/docs/tutorials/)   
[How-to-Guides](https://python.langchain.com/docs/how_to/)   

# 2 | Überblick Konzepte
---




Die Darstellung veranschaulicht das Grundprinzip von LangChain: die Verkettung verschiedener Konzepte zu einem durchgängigen Verarbeitungsprozess, der Eingaben in strukturierte Ausgaben umwandelt.

![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/langchain_prozess_01.png)

Konzepte von LangChain sind grundlegende Bausteine und Prinzipien. Hier sind einige der wichtigsten Konzepte:

+ Nachrichten (**Messages**): Kommunikationseinheiten in Chat-Modellen für Ein- und Ausgabe.
+ Prompt-Vorlagen (**Prompt template**s): Komponenten zur Erstellung strukturierter Prompts für LLMs.
+ Chat-Modelle (**Chat models**): LLMs, die über eine Chat-API verfügbar sind und Sequenzen von Nachrichten verarbeiten.
+ Chains (**Chains**): Verknüpfungen mehrerer LLMs oder anderer Komponenten für komplexere Anwendungen.
+ Strukturierte Ausgabe (**Structured output**): Technik, um Chat-Modelle in strukturierten Formaten antworten zu lassen.
+ Chat-Verlauf (**Chat history**): Eine Sequenz von Nachrichten, die eine Konversation darstellt.
+ Retrieval Augmented Generation (**RAG**): Technik zur Verbesserung von Sprachmodellen durch Kombination mit externen Wissensbasen.
+ Retriever (**Retriever**): Komponenten, die relevante Dokumente aus einer Wissensbasis abrufen.
+ Agenten (**Agents**): Nutzen Sprachmodelle, um Aktionssequenzen auszuwählen und mit externen Ressourcen zu interagieren.
+ Tools (**Tools**): Funktionen mit definierten Schemata für Name, Beschreibung und Argumente.


# 3 | Nachrichten-Typen
---

Beim Einsatz von Large Language Models (LLMs) wie GPT gibt es drei Hauptarten von Nachrichten (Messages), die die Interaktion mit dem Modell steuern. Diese Messages definieren, wie das Modell reagiert, welche Eingaben es erhält und welche Ausgaben es generiert.



**System Message:**   
Die `System Message` dient dazu, das **Verhalten des LLMs** zu steuern. Sie ist eine nicht sichtbare Nachricht, die dem Modell Anweisungen gibt, wie es sich verhalten soll. Dabei kann sie die Persönlichkeit, den Tonfall und die Einschränkungen des Modells beeinflussen. Die Systemnachricht funktioniert wie eine Art "Rollenspiel-Anweisung", um das Modell in eine gewünschte Rolle zu versetzen.

*Merkmale:*
- Definiert, wie das Modell reagieren soll
- Legt Persönlichkeit, Verhalten und Einschränkungen fest
- Wird zu Beginn einer Sitzung gesetzt und bleibt bestehen
- Nicht sichtbar für den Benutzer

*Beispiel einer System Message:*
```json
{
  "role": "system",
  "content": "Du bist ein KI-Assistent, der präzise und informative Antworten in einem professionellen Ton liefert."
}
```
Dieses Beispiel weist das Modell an, in einem professionellen Tonfall kurze und informative Antworten zu geben.

---



**User Message**   
Die `User Message` ist die eigentliche **Eingabe des Benutzer**s. Sie kann eine Frage, eine Aufforderung, kontextrelevanten Inhalt oder ein Befehl sein. Das Modell nutzt diese Nachricht als Ausgangspunkt für die Generierung einer Antwort.

*Merkmale:*
- Direkte Eingabe des Benutzers
- Kann eine Frage, Aufforderung, kontextrelevanten Inhalt oder einen Befehl enthalten
- Basis für die Antwort des Modells

*Beispiel einer User Message:*
```json
{
  "role": "user",
  "content": "Was sind die wichtigsten Unterschiede zwischen KI und maschinellem Lernen?"
}
```
Hier fragt der Benutzer nach den Unterschieden zwischen KI und maschinellem Lernen.

---


**AI Message**   
Die `AI Message` ist die **Antwort, die das Modell generiert**. Sie basiert auf der User Message und den Anweisungen aus der System Message. Die AI Message kann verschiedene Eigenschaften haben, ist aber hauptsächlich auf den Inhalt fokussiert.

*Merkmale:*
- Antwort des Modells auf die Benutzeranfrage
- Kann verschiedene Eigenschaften haben (z. B. Länge, Stil)
- Hauptsächlich inhaltlich relevant

*Beispiel einer AI Message:*
```json
{
  "role": "assistant",
  "content": "Künstliche Intelligenz (KI) ist ein weites Feld, das verschiedene Teilgebiete umfasst, darunter auch das maschinelle Lernen (ML). ML konzentriert sich auf die Erstellung von Modellen, die aus Daten lernen und Vorhersagen treffen."
}
```
Hier gibt das Modell eine inhaltliche Antwort auf die Frage des Benutzers.

---



**Kurz gefasst**   

+  **System Message** legt die Regeln und das Verhalten des Modells fest.
+ **User Message** stellt eine Anfrage oder einen Befehl.
+ **AI Message** gibt die generierte Antwort basierend auf den vorherigen Nachrichten.




# 4 | Prompts
---

![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/langchain_prozess_02.png)



Prompts sind die Eingaben bzw. Aufträge an das LLM. LangChain bietet verschiedene Template-Systeme. Ein **Template** ist eine **Vorlage** – genauer gesagt ein Text mit **Platzhaltern**, die später durch echte Werte ersetzt werden.


Ein **PromptTemplate** ist eine Vorlage für einen Eingabetext, der an ein Sprachmodell geschickt wird.
Er enthält Platzhalter `{placeholder}`, die später mit Nutzereingaben oder Kontext befüllt werden.

**Templates machen die KI-Prompts:**

+ Wiederverwendbar (z. B. für viele Themen oder Fragen)

+ Strukturiert (z. B. mit Rollen und Anweisungen)

+ Dynamisch erweiterbar (mit Memory, Tools, etc.)

## 4.1 | Modellbildung


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

In [ ]:
model_name = "gpt-4o-mini"
temperature = 0.0

**Standardwerte für den Baustein ChatOpenAI:**  

| Parameter                                     | Kurzbeschreibung                                                                        |
| --------------------------------------------- | --------------------------------------------------------------------------------------- |
| **model_name: 'gpt-3.5-turbo'**               | Bezeichnet das zu verwendende Sprachmodell (z. B. GPT-Version)                          |
| **temperature: 0.7**                          | Steuert die Kreativität der Antworten (niedriger = präziser, höher = freier)            |
| **max_tokens: None**                          | Kein festes Token-Limit; Ausgabe kann beliebig lang sein (bis zum Modellmaximum)        |
| **max_retries: 6**                            | Anzahl automatischer Wiederholungsversuche bei API-Fehlern                              |
| **n: 1**                                      | Gibt an, wie viele Antwortvarianten pro Anfrage generiert werden                        |
| **streaming: False**                          | Wenn *True*, werden Tokens beim Generieren gestreamt (z. B. für Live-Ausgabe)           |
| **verbose: False**                            | Wenn *True*, werden zusätzliche Debug-Informationen in der Konsole ausgegeben           |
| **openai_api_key: env('OPENAI_API_KEY')**     | Zugriffsschlüssel wird aus der Umgebungsvariable **OPENAI_API_KEY** gelesen             |





In [ ]:
# Modell definieren
llm = ChatOpenAI(model=model_name, temperature=temperature)

## 4.2 | Simple Prompt (ohne Rollen)


<p><font color='darkblue' size="4">
<b>❗Hinweis:</b>
</font></p>
Ohne Memory-Management hat das Modell ein kurzes Gedächnis.


**Prompt wird als Zeichenkette übergeben:**

In [ ]:
# Aufruf
response = llm.invoke("Was ist Generative KI?")

In [ ]:
# Antwort
type(response)

In [ ]:
response

In [ ]:
for r in response:
    print(r)

In [ ]:
# Ausgabe
mprint("## 📣 Model response:")
mprint("---")
mprint(response.content)

In [ ]:
# Aufruf
response = llm.invoke("Was war meine letzte Frage?")

In [ ]:
# Ausgabe
mprint("## 📣 Model response:")
mprint("---")
mprint(response.content)

**Prompt wird als formatierte String-Variable übergeben:**

In [ ]:
# Pure Python Lösung für simple prompt mit template
thema = "Machine Learning"
prompt = f"Erkläre {thema} in einfachen Worten."

In [ ]:
# Aufruf
response = llm.invoke(prompt)

In [ ]:
# Ausgabe
mprint("## 📣 Model response:")
mprint("---")
mprint(response.content)

**Prompt wird als template übergeben:**

In [ ]:
# Einfaches Template mit Platzhaltern
simple_prompt_template = PromptTemplate.from_template(
    "Erkläre {thema} in einfachen Worten."
)

In [ ]:
# Prompt-Erstellung
thema = "Generative KI"
prompt = simple_prompt_template.format(thema=thema)
prompt

In [ ]:
# Aufruf
response = llm.invoke(prompt)

In [ ]:
# Ausgabe
mprint("## 📣 Model response:")
mprint("---")
mprint(response.content)

**Vorteil PromptTemplate vs f-string**

| Kriterium                        | `f-string`                | `PromptTemplate`                         |
| -------------------------------- | ------------------------- | ---------------------------------------- |
| 🧱 Basisfunktion                 | String zusammenbauen      | String-Vorlage mit Platzhaltern          |
| 🔁 Wiederverwendbarkeit          | manuell                   | sehr gut (Template + `.format()`)        |
| 🔌 Integration mit LangChain     | ❌ Nur als fertiger Text   | ✅ Nahtlos (in Chains, mit LLMs, etc.)    |
| 📦 Kombinierbar mit Tools        | ❌ Nein                    | ✅ Ja (Memory, OutputParser, Agents)      |
| 🧠 Erweiterbar (z. B. Beispiele) | ❌ Mühsam                  | ✅ z. B. `FewShotPromptTemplate`          |
| 🕵️ Nachvollziehbarkeit          | schwer bei großen Prompts | gut dokumentierbar & testbar             |
| 👥 Rollensteuerung               | ❌ Nur manuell             | ✅ mit `ChatPromptTemplate`               |
| 🧪 Validierung                   | ❌ keine                   | ✅ Platzhalterprüfung (`input_variables`) |


## 4.3 | Chat Prompt (mit Rollen) 📌

Das **ChatPromptTemplate** ist eine spezielle Prompt-Vorlage für Chatmodelle (z. B. GPT-3.5, GPT-4), die **mehrere Rollen und Nachrichten** unterstützt – also genau das, was Chat-Modelle eigentlich brauchen.

Es gehört zur LangChain-Bibliothek und baut auf dem Prinzip auf:


**Wer sagt was? → system, user, assistant, etc.**

In [ ]:
# prompt-template als tuple
chat_template = ChatPromptTemplate([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("human", "Erkläre mir {thema}"),
])

**... oder ...**

In [ ]:
# prompt-template als dict
chat_template = ChatPromptTemplate([
    {"role": "system", "content": "Du bist ein hilfreicher und humorvoller Assistent."},
    {"role": "human", "content": "Erkläre mir {thema}"},
])

**... oder ...**

In [ ]:
# Prompt-Template als Nachrichtenobjekte
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("human", "Erkläre mir {thema}")
])

In [ ]:
# Thema definieren
thema = "Machine Learning"

# Prompt formatieren - hier die richtige Methode verwenden
prompt = chat_template.format_messages(thema=thema)

In [ ]:
# Ausgabe der formatierten Nachrichten
for message in prompt:
    print(f"Role: {message.type}, Content: {message.content}")

In [ ]:
# Aufrufen der Kette mit Eingaben
response = llm.invoke(prompt)

In [ ]:
# Ausgabe
mprint("## 📣 Model response:")
mprint("---")
mprint(response.content)

**Vorteile gegenüber `PromptTemplate`**

| Vorteil                      | `PromptTemplate`     | `ChatPromptTemplate`            |
| ---------------------------- | -------------------- | ------------------------------- |
| 🚻 Rollenstruktur            | ❌ Nur einfacher Text | ✅ Klar: `system`, `user`, `ai`  |
| 📚 Konversationsaufbau       | ❌ Nicht geeignet     | ✅ Ideal für Multi-Turn-Dialoge  |
| 🔄 Kombinierbar mit `Memory` | ❌ Nein               | ✅ Ja, via `MessagesPlaceholder` |
| 🔌 Für Chatmodelle wie GPT   | ⚠️ Nur als Fließtext | ✅ Nativ & strukturiert          |
| 🔍 Klarheit & Wartbarkeit    | ⚠️ Nur Inline-Text   | ✅ Saubere Message-Trennung      |
| 🧪 Prompt-Testbarkeit        | ❌ Eingeschränkt      | ✅ Besser strukturiert           |


## 4.4 | Few-Shot-Learning Prompt


Few-Shot-Prompting ist eine Technik, bei der man einem KI-Modell einige Beispiele (sogenannte "Shots") vorlegt, bevor es eine neue Aufgabe lösen soll. Das Modell kann aus diesen Beispielen das Muster erkennen und auf neue, ähnliche Fälle anwenden.

In [ ]:
examples = [
    {"frage": "Was ist Python?", "antwort": "Eine interpretierende Programmiersprache."},
    {"frage": "Was ist Java?", "antwort": "Eine objektorientierte Sprache."}
]

In [ ]:
# Few-Shot-Learning als f-String
few_shot_prompt = f"""
Du bist ein Assistent, der Fragen zu Programmiersprachen beantwortet. Halte deine Antworten kurz und präzise.

Beispiel 1:
Frage: {examples[0]['frage']}
Antwort: {examples[0]['antwort']}

Beispiel 2:
Frage: {examples[1]['frage']}
Antwort: {examples[1]['antwort']}

Beantworte nun die folgende Frage im gleichen Stil:
Frage: {{frage}}
Antwort:
"""

In [ ]:
frage = "Was ist Rust?"
prompt = few_shot_prompt.format(frage=frage)

In [ ]:
response = llm.invoke(prompt)

In [ ]:
mprint("## 📣 Model response:")
mprint("---")
mprint(response.content)

**... oder ...**

`FewShotPromptTemplate` ist eine Prompt-Vorlage, die automatisch mehrere Beispiele einfügt, bevor das eigentliche Nutzer-Input kommt.

In [ ]:
# Prompt für einzelne Beispiele
example_prompt = PromptTemplate.from_template(
    "Frage: {frage}\nAntwort: {antwort}"
)

In [ ]:
# Few-Shot-Prompt
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Du bist ein Assistent, der Fragen zu Programmiersprachen beantwortet. Halte deine Antworten kurz und präzise:",
    suffix="Frage: {frage}\nAntwort:",
    input_variables=["frage"]
)

prompt = few_shot_prompt.format(frage="Was ist Go?")

In [ ]:
response = llm.invoke(prompt)

In [ ]:
mprint("## 📣 Model response:")
mprint("---")
mprint(response.content)

**Vorteile eines `FewShotPromptTemplate`**

| Vorteil                      | Beschreibung                                                      |
| ---------------------------- | ----------------------------------------------------------------- |
| 🧠 Lerneffekt für das Modell | Das Modell erkennt aus Beispielen, wie es antworten soll          |
| 🧰 Flexibilität              | Beispiele können dynamisch gesetzt oder generiert werden          |
| 🧱 Struktur & Konsistenz     | Einheitlicher Aufbau aller Beispiele über `example_prompt`        |
| 🪄 Kein Training nötig       | Kein Fine-Tuning – nur durch Prompting                            |
| 🛠 Kombinierbar              | Kann mit Templates, Tools, Chains, OutputParser kombiniert werden |


# 5 | Modelle
---

LangChain ist mit zahlreichen LLMs – sowohl eines als auch verschiedener Anbieter – kompatibel und bietet vielseitige Konfigurationsoptionen für individuelle Anwendungsfälle.

**Beispiel `gpt-4o-mini` und `o3-mini`**

+ gpt-4o-mini ist ein kompaktes, multimodales KI-Modell von OpenAI, das Text- und Bildeingaben verarbeiten kann und besonders für schnelle, alltägliche Aufgaben sowie kleinere Anwendungen optimiert wurde

+ o3-mini ist ein spezialisiertes KI-Modell, das vor allem für komplexe Wissensverarbeitung, anspruchsvolle Denkaufgaben und professionelle Programmierprojekte entwickelt wurde

**Modellparameter:**

| **Parameter**       | **`gpt-4o`**     | **`gpt-4o-mini`** | **`o3-mini`**    | **Erklärung**                                                               |
| ------------------- | ---------------- | ----------------- | ---------------- | --------------------------------------------------------------------------- |
| `model_name`        | ✅                | ✅                 | ✅                | Modellbezeichnung, z. B. `"gpt-4o"`                                         |
| `temperature`       | ✅                | ✅                 | ❌                | Kreativität/Zufall (0.0 = deterministisch, 1.0 = kreativ)                   |
| `top_p`             | ✅                | ✅                 | ❌                | Nucleus Sampling (Alternative zu `temperature`)                             |
| `max_tokens`        | ✅                | ✅                 | ✅                | Maximale Anzahl an generierten Tokens                                       |
| `frequency_penalty` | ✅                | ✅                 | ❌                | Bestraft Wiederholungen im Output                                           |
| `presence_penalty`  | ✅                | ✅                 | ❌                | Fördert neue Inhalte statt Wiederholungen                                   |
| `streaming`         | ✅                | ✅                 | ✅                | Ausgabe erfolgt tokenweise in Echtzeit (Streaming API)                      |
| `seed`              | ✅ *(via kwargs)* | ✅ *(via kwargs)*  | ✅ *(via kwargs)* | Fixiert Zufallsverhalten für reproduzierbare Ergebnisse                     |
| `response_format`   | ✅ *(via kwargs)* | ✅ *(via kwargs)*  | ✅ *(via kwargs)* | Gibt Ausgabeformat wie `"text"` oder `"json"` vor                           |
| `logit_bias`        | ✅ *(via kwargs)* | ✅ *(via kwargs)*  | ✅ *(via kwargs)* | Gewichtung einzelner Tokens zur Output-Steuerung                            |
| `tool_choice`       | ✅ *(via kwargs)* | ✅ *(via kwargs)*  | ✅ *(via kwargs)* | Auswahl eines bestimmten Tools beim Function Calling                        |
| `reasoning_effort`  | ❌                | ❌                 | ✅                | o3-spezifisch: `"low"`, `"medium"`, `"high"` zur Steuerung des Denkaufwands |



In [ ]:
# Modelle konfigurieren

# Chat-Modell
chat_model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.9
)

# Reasoning-Modell
resoning_model = ChatOpenAI(
    model_name="o3-mini",
)

In [ ]:
# Gemeinsamer simple prompt
prompt = "Was ist der Unterschied zwischen Prompting und Retrieval-Augmented-Generation?"

In [ ]:
# Aufruf
response = chat_model.invoke(prompt)

In [ ]:
mprint("## 📣 Chat-Model:")
mprint("---")
mprint(response.content)

In [ ]:
# Aufruf
response = resoning_model.invoke(prompt)

In [ ]:
mprint("## 📣 Reasoning-Model:")
mprint("---")
mprint(response.content)

**📊 Vergleich: Chat-Modell vs. Reasoning-Modell**

| **Aspekt**                      | **Chat-Modell**                                                           | **Reasoning-Modell**                                                                                |
| ------------------------------- | ------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------- |
| **Stil**                        | Erklärend, klassisch strukturiert mit Fließtext                           | Punktuell, technisch und systematisch                                                               |
| **Formatierung**                | Fließtext mit Absätzen und Beispielen                                     | Aufzählungen (•) und strukturierte Unterpunkte                                                      |
| **Definition Prompting**        | Kurze Definition als Text, danach Beispiel                                | Technisch klar: „parametrisches Wissen“, Rolle der Prompt-Formulierung betont                       |
| **Definition RAG**              | Fließtext-Erklärung mit Ablaufbeschreibung und Beispiel                   | Detaillierte Schritte: Abruf, Kontextintegration, Vorteile bzgl. Aktualität und Präzision erläutert |
| **Beispiele**                   | Ja – einfache, praxisnahe Beispiele (z. B. Klimawandel, Quantencomputing) | Nein – keine konkreten Beispiele, aber kontextbezogene Beschreibung                                 |
| **Vergleich Prompting vs. RAG** | Abstrakte Gegenüberstellung in einem abschließenden Absatz                | Direkter Vergleich im letzten Abschnitt mit Betonung der Wissensquellen                             |
| **Technische Tiefe**            | Mittel – für Einsteiger\:innen gut verständlich                           | Hoch – Begriffe wie „parametrisches Wissen“, Fokus auf Architektur                                  |
| **Zielgruppe**                  | Allgemeines Publikum, Einstieg in das Thema                               | Fortgeschrittene Nutzer\:innen, Technik-affine Leserschaft                                          |
| **Stärken des Modells**         | Klare Erklärung, gute didaktische Struktur                                | Präzise Begriffsabgrenzung, hohe Informationsdichte                                                 |



**📊 Vergleich: Chat-Modelle**

| Model              | Creator    | Open?  | API Name(s) (falls verfügbar)                            | Context Window             |
| ------------------ | ---------- | ------ | -------------------------------------------------------- | -------------------------- |
| **GPT-5**          | OpenAI     | Closed | `gpt-5`, `gpt-5-mini`, `gpt-5-nano`, `gpt-5-chat-latest` | 400K Tokens (~296 Seiten)  |
| **GPT-4.1**        | OpenAI     | Closed | `gpt-4.1`, `gpt-4.1-mini`, `gpt-4.1-nano`                | 1M Tokens (~741 Seiten)    |
| **GPT-4o**         | OpenAI     | Closed | `gpt-4o`, `gpt-4o-mini`                                  | 128K Tokens (~95 Seiten)   |
| **GPT-4 Turbo**    | OpenAI     | Closed | `gpt-4-turbo`, `gpt-4-turbo-2024-04-09`                  | 128K Tokens (~95 Seiten)   |
| **gpt-oss-120b**   | OpenAI     | Open   | —                                                        | 131K Tokens (~97 Seiten)   |
| **gpt-oss-20b**    | OpenAI     | Open   | —                                                        | 131K Tokens (~97 Seiten)   |
| **Claude 4**       | Anthropic  | Closed | `claude-sonnet-4-20250514`, `claude-opus-4-20250514`     | 200K Tokens (~148 Seiten)  |
| **Gemini 2.5 Pro** | Google     | Closed | `gemini-2.5-pro`, `gemini-2.5-pro-exp-03-25`             | 1M Tokens (~741 Seiten)    |
| **Llama 3.1 405B** | Meta       | Open   | —                                                        | 512K Tokens (~379 Seiten)  |
| **DeepSeek-V3**    | DeepSeek   | Open   | —                                                        | 128K Tokens (~95 Seiten)   |
| **Mistral 7B**     | Mistral.AI | Open   | —                                                        | 32K Tokens (~24 Seiten)    |

<br>

*Umrechnung Token/Seiten:  ~1.350 Tokens ≈ 1 Seite deutschsprachiger Text*

# 6 | Chains
---

Chains verbinden mehrere Komponenten zu einer Verarbeitungskette. LangChain Expression Language (LCEL) ist das **neue Programmiermodell in LangChain v0.3+**, das die Entwicklung von LLM-Anwendungen vereinfacht.



**Hier die Kernpunkte:**

1. Was ist LCEL?
```python
# LCEL nutzt den Pipe-Operator (|) für klare Verkettungen
chain = prompt | model | parser
```
- Eine deklarative Sprache zum Verketten von LangChain-Komponenten
- Ermöglicht linearen Datenfluss zwischen Komponenten
- Basiert auf dem Pipe-Operator (|) für intuitive Verbindungen

2. Warum LCEL nutzen?
- Bessere Lesbarkeit des Codes
- Einfachere Wartung und Debugging
- Verbesserte Performance durch optimierte Ausführung
- Bessere Typsicherheit und Fehlererkennung
- Unterstützt modernes Streaming und Async-Operationen

3. Praktisches Beispiel:

## 6.1 | Simple Chain 📌


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. Prompt-Template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("human", "{input}")
])

# 2. Modell
model_name = "gpt-4o-mini"
temperature = 0

llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature
)

# 3. Parser
parser = StrOutputParser()

#  4. Einfache LCEL-Kette
chain = prompt | llm | parser

# 5. Ausführung
response = chain.invoke({"input": "Erkläre LangChain Expression Language."})

In [ ]:
mprint("## 📣 Model response:")
mprint("---")
mprint(response)


LCEL ist der empfohlene Weg für alle neuen LangChain-Projekte, da es die Entwicklung vereinfacht und zukunftssicher macht.

## 6.2 | Sequential Chains


Sequentielle Chain: Linear, ein Input wird schrittweise verarbeitet

In [ ]:
# Prompts
zusammenfassen_template = ChatPromptTemplate.from_messages([
    ("system", "Fasse den folgenden Text prägnant zusammen."),
    ("human", "{text}")
])

uebersetzen_template = ChatPromptTemplate.from_messages([
    ("system", "Übersetze den folgenden Text ins Deutsche."),
    ("human", "{text}")
])

In [ ]:
# Verarbeitungskette

# Erste Chain: Zusammenfassung
summarization_chain = (
    zusammenfassen_template
    | chat_model
    | (lambda output: {"text": output})  # Notwendig, für String-> Dict, wird von translation_chain erwartet
)

# Zweite Chain: Translation/Übersetzung
translation_chain = (
    uebersetzen_template
    | chat_model
)

# Sequential Chain: Zusammenfassen und dann Übersetzen
sequential_chain = summarization_chain | translation_chain | parser

In [ ]:
# Input Text
user_input = """
Machine Learning is a specialized branch of artificial intelligence (AI) dedicated to the development of algorithms and systems that can automatically learn from data and past experiences. Instead of following rigid, pre-defined instructions, machine learning models identify patterns, adapt their behavior, and improve their performance over time based on the information they are exposed to. This ability enables them to make predictions, recognize complex relationships, and solve problems in dynamic environments. Machine learning techniques are widely applied across various fields, including healthcare, finance, transportation, and entertainment, driving innovations such as personalized recommendations, autonomous vehicles, and intelligent diagnostics.
"""

In [ ]:
# Aufruf der Kette
result = sequential_chain.invoke({"text": user_input})

In [ ]:
mprint("## 📣 Model response:")
mprint("---")
mprint(result)

## 6.3 | Q&A-Chain


QA Chain: Kontextbasiert, kombiniert mehrere Inputs für eine einzelne Verarbeitung

In [ ]:
# Q&A Prompt
qa_template = ChatPromptTemplate.from_messages([
    ("system", "Beantworte die Frage basierend auf dem gegebenen Kontext."),
    ("human", """
    Kontext: {context}

    Frage: {question}
    """)
])

In [ ]:
# Verkettung
qa_chain = qa_template | chat_model | parser

In [ ]:
# Inhalte
context = """
Python ist eine beliebte Programmiersprache für Machine Learning.
Sie bietet viele Bibliotheken wie TensorFlow und PyTorch.
"""
question = "Welche ML-Bibliotheken gibt es für Python?"

In [ ]:
# Aufruf
result = qa_chain.invoke({
    "context": context,
    "question": question
})

In [ ]:
mprint("## 📣 Model response:")
mprint("---")
mprint(result)

# 7 | OutputParser 📌
---

![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/langchain_prozess_03.png)

Ein OutputParser ist ein Konzept in LangChain, das hilft, die Antworten eines Modells richtig weiterzuverarbeiten. Es nimmt die **rohe Ausgabe** (zum Beispiel einen langen Text) und formt sie in ein **bestimmtes Format**, das später in der Anwendung leichter benutzt werden kann.

In [ ]:
from langchain_core.output_parsers.string import StrOutputParser

# Einfacher Prompt ohne explizite Formatvorgabe
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent. Formatiere Deine Antwort im json-Format mit den Infos zu Name und Alter."),
    ("human", "{user_input}")
])


model_name = "gpt-4o-mini"
temperature	= 0

# Modell
llm = ChatOpenAI(model=model_name, temperature=temperature)

# Struktur der Antwort definieren
parser = StrOutputParser()

# LCEL-Chain
chain = prompt | llm | parser

In [ ]:
# Ausführen
response = chain.invoke({"input": "Bitte gibt den Namen einer Person und das Alter"})

In [ ]:
# Ausgabe
mprint("## 📣 Model response:")
mprint("---")
mprint(response)

# 8 | Runnables
---


![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/langchain_prozess_01.png)

Ein **Runnable** ist ein elementarer Baustein in LangChain. Er nimmt eine Eingabe und liefert eine Ausgabe - wie eine Funktion mit klaren Regeln. Man kann sich Konzepte wie Prompts, LLMs oder Parser als Runnables vorstellen. Runnable kann man als **Oberbegriff** für Prompts, LLMs, Parser und ähnliche Komponenten verwenden.



In der Grafik sieht man die **drei wichtigsten Runnables**: Prompt-Template, LLM und Parser, die zusammen eine Kette bilden.

In LangChain sind die wichtigsten Komponenten als Runnables verfügbar:

+ Prompts/Templates (der erste Baustein in der Kette)
+ LLMs (das Herzstück in der Mitte)
+ Output-Parser (verwandelt die LLM-Antwort in ein nutzbares Format)
+ Komplette Chains (die ganze Verarbeitungskette)
+ Tools (für spezielle Aufgaben)
+ Retriever (holen zusätzliche Informationen)


Vorteile von Runnables:

+ Gleiche Bedienung: Alle Runnables (Prompt, LLM, Parser) funktionieren nach demselben Prinzip
+ Einfaches Verbinden: Man kann sie unkompliziert zu einer Kette zusammenfügen - genau wie im Bild gezeigt
+ Anpassungsfähig: Sie funktionieren sowohl einzeln als auch in der Gruppe
Schrittweise Ausgabe: Ergebnisse können stückweise weitergegeben werden

<p><font color='black' size="5">
Benutzerdefinierte Runnables erstellen
</font></p>

Man kann auch eigene Runnables erstellen, indem man eine Klasse definiert, die das Runnable-Interface implementiert oder eine Funktion mit `RunnableLambda` umsetzt:


In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers.string import StrOutputParser

# 1. Mit RunnableLambda eine einfache Transformation erstellen
def text_verdoppeln(eingabe):
    return f"{eingabe} <br> <br>  {eingabe}"

verdoppler = RunnableLambda(text_verdoppeln)

# Verwendung des eigenen Runnables
verdoppler.invoke("Dies ist ein Text.")

In [ ]:
# Einfacher Prompt ohne explizite Formatvorgabe
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent. Antworte kurz."),
    ("human", "Eingabe: {input}")
])

# In eine Kette einbauen - Korrektur der Pipe-Operatoren
chain_mit_verdoppler = prompt | llm | parser | verdoppler

# Test der Kette
response = chain_mit_verdoppler.invoke({"input": "Wer war Einstein?"})

In [ ]:
mprint("## 📣 Model response:")
mprint("---")
mprint(response)


<p><font color='black' size="5">
Runnable-Methoden
</font></p>

Jedes Runnable unterstützt standardmäßig folgende Methoden:

1. **invoke()**: Für einzelne, synchrone Anfragen
2. **batch()**: Verarbeitet mehrere Eingaben parallel
3. **stream()**: Gibt Teilergebnisse zurück, sobald sie verfügbar sind
4. **ainvoke()**: Asynchrone Version von invoke
5. **abatch()**: Asynchrone Version von batch
6. **astream()**: Asynchrone Version von stream



Hier ein einfaches Beispiel mit den verschiedenen Aufrufmethoden:


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

# Einfaches Template erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("human", "{input}")
])

# Runnable-Kette erstellen
chain = prompt | llm | parser

In [ ]:
# 1. invoke - Einzelne Anfrage
response = chain.invoke({"input": "Was ist ein Runnable in LangChain?"})

mprint("## 📣 Invoke response:")
mprint("---")
mprint(response)

In [ ]:
# 2. batch - Mehrere Anfragen parallel
response = chain.batch([
    {"input": "Erkläre den Begriff LCEL kurz."},
    {"input": "Was sind die Hauptvorteile von Runnables?"}
])
mprint("##  📣 Batch response::")
mprint("---")
for i, ergebnis in enumerate(response, 1):
    mprint(f"Anfrage {i}: {response[:50]}...\n")

In [ ]:
# 3. stream - Schrittweise Ausgabe
mprint("##  📣 Stream response:")
mprint("---")
for chunk in chain.stream({"input": "Erkläre die Abkürzung LCEL kurz."}):
    print(chunk, end=" ➡ ")  # Zeichen trennt die via stream gelieferten Ergebnisse/Ausgaben
print("\n")

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
Erstellen einer LangChain-Kette zur Textanalyse
</font></p>

Entwickeln Sie eine Verarbeitungskette, die einen Text analysiert und verschiedene Informationen darüber extrahiert.

**Schritte:**
1. Erstelle ein ChatPromptTemplate, das ein LLM anweist, einen Text zu analysieren
2. Die Analyse soll folgende Aspekte umfassen:
   - Hauptthema des Textes
   - Tonalität (formal, informell, etc.)
   - Schlüsselwörter (5-10)
   - Kurze Zusammenfassung (max. 3 Sätze)
3. Formatiere die Ausgabe strukturiert mit Markdown-Überschriften
4. Teste die Kette mit mindestens zwei verschiedenen Texten




<p><font color='black' size="5">
Few-Shot-Learning für Textklassifikation
</font></p>

Erstellen Sie ein System, das mithilfe von Few-Shot-Learning Texte in vorgegebene Kategorien klassifiziert.

**Schritte:**
1. Definiere 3-5 Kategorien für die Klassifikation (z.B. Sport, Politik, Technologie, Kultur)
2. Erstelle einen FewShotPromptTemplate mit Beispielen für jede Kategorie
3. Entwickle eine Chain, die neue Texte klassifiziert
4. Implementiere eine Funktion, die neben der Kategorie auch eine Begründung für die Einordnung liefert
5. Teste das System mit verschiedenen Texten, die nicht in den Beispielen vorkommen


<p><font color='black' size="5">
Q&A-System mit Sequential Chain
</font></p>

Entwickeln Sie ein System, das Fragen zu einem gegebenen Kontext beantwortet, aber zuerst den Kontext zusammenfasst und dann die Frage beantwortet.


**Schritte:**

+ Erstellen Sie zwei Templates:
    + Ein Template zur Zusammenfassung des Kontexts
    + Ein Template zur Beantwortung einer Frage basierend auf der Zusammenfassung
+ Verbinden Sie die Templates in einer sequentiellen Kette
+ Implementieren Sie einen Mechanismus, der die Zusammenfassung und die Antwort getrennt zurückgibt
+ Nutzen Sie LCEL (LangChain Expression Language) für die Verkettung
+ Testen Sie das System mit verschiedenen Kontexten und Fragen